In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
df = pd.read_csv('成都写字楼数据.csv')

In [6]:
def replacestr(string):
    string = str(string).replace('[','')
    string = string.replace(']','')
    string = string.replace("'",'')
    return string

df['loupandizhi'] = df['loupandizhi'].apply(replacestr)

list_location = df['loupandizhi'].value_counts().index.tolist()

dict_location = {}
Key = '6b95b84cab6100c0a6f52c62ece97e4b'
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in list_location:
    location = i.split(",")[-1].strip()
    url = url_pre.format(Key, location, '成都')
    res = requests.get(url).json()
    if res['status'] == '0':
        dict_location[i] = np.nan
        counter +=1
    else:
        try: 
            dict_location[i] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            dict_location[i] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed


In [7]:
df['location_coords'] = np.nan

counter = 0
for i in list_location:
    counter += 1
    df_temp = df[df['loupandizhi'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,14] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [8]:
df.head(3)

,building_type,dengji,jianzhumianji,loupan_content,loupandizhi,price_perunit,release_date,service_price,suozailouceng,titile,total_price_10K,url,xzl_content,zhuangxiu,location_coords
0,毛坯(可分割),甲级,1560㎡,项目总占地约48亩，总建筑面积16万方，由大型综合性商业卖场、甲级写字楼、SOHO公寓、酒店...,"成都, 成华, 成渝立交, 中国铁建广场",(12500元/㎡),2019/8/5 10:37:18,6.00元/平米·月,高区(共30层),中国铁建广场 东2.5环 双地铁站 央企打造 产权清晰 可按揭,1950,https://cd.office.fang.com/shou/3_206435594.html,NaN,NaN,"104.142387,30.679966"
1,纯写字楼,NaN,919.9㎡,国航世纪中心雄踞人南核心，磅礴大气尽显国际重磅企业身份，中心地位！ \r\n该项目是由中国航...,"成都, 武侯, 桐梓林, 国航世纪中心",(80元/㎡),2019/9/5 21:11:34,NaN,\r\n,地铁口物业 桐梓林 精装带全套家具 国航世纪中心,7,https://cd.office.fang.com/shou/3_208846821.html,=房屋简介= 【国航世纪中心】1、房屋面积：919.9㎡2、租金单价： 80元/㎡/月...,精装修(可分割),"104.068047,30.616817"
2,商业综合体楼,甲级,1100㎡,项目总占地面积466667.05㎡总建筑面积约176万㎡（其中新世纪环球中心约150万㎡ 当...,"成都, 高新区, 新会展, 新世纪环球中心",(12000元/㎡),2019/8/15 18:04:28,19.00元/平米·月,中区(共21层),1100㎡环球中心 豪装带家具1100平有产权,1320,https://cd.office.fang.com/shou/3_208600117.html,· 您花大量时间在网上到处看房，您要是信得过我小罗（罗杨），就直接给我来个电话，告诉我您的需...,精装修(可分割),"104.063392,30.568751"


In [12]:
df.shape

(3929, 15)

In [11]:
df.to_csv('成都写字楼地址数据.csv', encoding = 'utf-8-sig',index = False)